<a href="https://colab.research.google.com/github/kylemath/DelCloseBot/blob/master/notebooks/DelCloseBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DelCloseBot

Goal is to make a DelCloseBot using old youtube footage. Will have live actor filmed with webcam during show whose pose gets transformed into real time DelClose Poses shown to audience, with voice impressonation reading lines scripted from dialogue AI. 


*   Using https://github.com/datitran/face2face-demo
*   youtube video here https://www.youtube.com/watch?v=DjbQaSPPsqA
*   saved into .mp4 with https://www.onlinevideoconverter.com/youtube-converter 



Instructions:
* Connect colab to google drive 
* save the .mp4 video into a folder, 
* download the facial landmark model from http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
* after uncompressing locally and saving to google drive 

Tactics:
* Convert video into side by side images of video and face structure
* Train model on these side by side images to predict DelClose from Face pose
* Use live webam to get net face structure
* Convert new face structure to DelClose pose with model in real time

**Instal git repo**

In [2]:
!git clone https://github.com/datitran/face2face-demo.git --recursive
%cd face2face-demo


Cloning into 'face2face-demo'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 32
Unpacking objects: 100% (33/33), done.
/content/face2face-demo


**Install anaconda and setup to use**

In [3]:
!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')


--2019-01-13 06:32:45--  https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.16.19.10, 104.16.18.10, 2606:4700::6810:120a, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.16.19.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 577996269 (551M) [application/x-sh]
Saving to: ‘Anaconda3-5.1.0-Linux-x86_64.sh’

Anaconda3-5.1.0-Lin 100%[===================>] 551.22M   120MB/s    in 4.5s    

2019-01-13 06:32:50 (122 MB/s) - ‘Anaconda3-5.1.0-Linux-x86_64.sh’ saved [577996269/577996269]

PREFIX=/usr/local
installing: python-3.6.4-hc3d631a_1 ...
Python 3.6.4 :: Anaconda, Inc.
installing: ca-certificates-2017.08.26-h1d4fec5_0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: intel-openmp-2018.0.0-hc7b2577_8 ...
installing: libgcc-ng-7.2.0-h7cc24e2_2 ...
installing: libgfortran-ng-7.2.0-h9f7466a_2 ...
installing: libstdcxx-ng-7.2.0-h7a57d05_2 ...
installing: bzip2-1.0.6-h9a117a8_4

** Create conda environment and activate it **

In [4]:
!conda env create -f environment.yml 
!source activate face2face-demo

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

** Install some needed packages **

ISSUE - dlib install takes about 10 minutes to compile, find way to speed up?

In [18]:
!pip install opencv-python
!pip install dlib
!pip install imutils
!pip install tensorflow


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [83.2 kB]
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release
Get:11 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [1,365 B]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [303 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Pack

**Mount your google drive to load and save from**


Note you have to follow the link here and get the activation code before proceeding

In [19]:
#mount google drive to load own images, follow link and get code and paste in below
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


** Processes the source video to find frames with faces, and pull out the face structure, save side by side image **

In [73]:
!python generate_train_data.py \
  --file /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/DelCloseMono.mp4 \
  --num 400 \
  --landmark-model /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/shape_predictor_68_face_landmarks.dat

No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
1
[INFO] elapsed time: 0.08
No face detected
No face detected
No face detected
No face detected
2
[INFO] elapsed time: 0.08
No face detected
No face detected
No face detected
No face detected
3
[INFO] elapsed time: 0.08
4
[INFO] elapsed time: 0.08
5
[INFO] elapsed time: 0.08
6
[INFO] elapsed time: 0.08
7
[INFO] elapsed time: 0.08
8
[INFO] elapsed time: 0.08
No face detected
9
[INFO] elapsed time: 0.08
10
[INFO] elapsed time: 0.08
11
[INFO] elapsed time: 0.08
12
[INFO] elapsed time: 0.08
13
[INFO] elapsed time: 0.08
No face detected
No face detected
No face detected
14
[INFO] elapsed time: 0.08
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face 

In [74]:
# Clone the repo from Christopher Hesse's pix2pix TensorFlow implementation
%cd ..
!git clone https://github.com/affinelayer/pix2pix-tensorflow.git
 


/content


In [79]:
%cd ..

/content


In [80]:
# Move the original and landmarks folder into the google drive folder
!cp -r face2face-demo/landmarks -r face2face-demo/original /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos

# Go into the pix2pix-tensorflow folder
%cd pix2pix-tensorflow/


/content/pix2pix-tensorflow


In [81]:
# Resize original images
!python tools/process.py \
  --input_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/original \
  --operation resize \
  --output_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/original_resized
  
# Resize landmark images
!python tools/process.py \
  --input_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/landmarks \
  --operation resize \
  --output_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/landmarks_resized
  
# Combine both resized original and landmark images
!python tools/process.py \
  --input_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/landmarks_resized \
  --b_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/original_resized \
  --operation combine \
  --output_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/combined
  
# Split into train/val set
!python tools/split.py \
  --dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/combined

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
skipping 0 files that already exist
processing 400 files
2019-01-13 08:02:42.730753: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
1/400 complete  7.96 images/sec  0m0s elapsed  0m50s remaining
2/400 complete  9.73 images/sec  0m0s elapsed  0m40s remaining
3/400 complete  10.39 images/sec  0m0s elapsed  0m38s remaining
4/400 complete  10.86 images/sec  0m0s elapsed  0m36s remaining
5/400 complete  11.01 images/sec  0m0s elapsed  0m35s remaining
6/400 complete  11.14 images/sec  0m0s elapsed  0m35s remaining
7/400 complete  11.20 images/sec  0m0s elapsed  0m35s remaining
8/400 complete  11.

In [0]:
# Train the model on the data
!python pix2pix.py \
  --mode train \
  --output_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/face2face-model \
  --max_epochs 200 \
  --input_dir /content/gdrive/My\ Drive/DeepLearning/DelCloseBot/photos/combined/train \
  --which_direction AtoB


/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
aspect_ratio = 1.0
batch_size = 1
beta1 = 0.5
checkpoint = None
display_freq = 0
flip = True
gan_weight = 1.0
input_dir = /content/gdrive/My Drive/DeepLearning/DelCloseBot/photos/combined/train
l1_weight = 100.0
lab_colorization = False
lr = 0.0002
max_epochs = 200
max_steps = None
mode = train
ndf = 64
ngf = 64
output_dir = /content/gdrive/My Drive/DeepLearning/DelCloseBot/face2face-model
output_filetype = png
progress_freq = 50
save_freq = 5000
scale_size = 286
seed = 458449941
separable_conv = False
summary_freq = 100
trace_freq = 0
which_direction = AtoB
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(str

In [59]:
%cd ..
%cd face2face-demo


/content/face2face-demo


In [61]:
!python reduce_model.py --model-input ../pix2pix-tensorflow/face2face-model --model-output ../pix2pix-tensorflow/face2face-reduced-model

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
2019-01-13 06:31:32.905209: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Traceback (most recent call last):
  File "reduce_model.py", line 212, in <module>
    saver.restore(sess, checkpoint)
  File "/usr/local/lib/python3.6/site-packages/tensorflow/python/training/saver.py", line 1534, in restore
    raise ValueError("Can't load save_path when it is None.")
ValueError: Can't load save_path when it is None.
